### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Quran With Flashcards/Api Key/quran-with-flashcards-firebase-adminsdk-t8xhw-202e2c7561.json")  # Quran with Flashcards
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
df_28 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ne o beni ne de ben onu bir daha,3870,3873,bundan böyle ne o beni ne de ben onu bir daha ...,FUS0hsPCtMs,https://www.youtube.com/watch?v=FUS0hsPCtMs&t=...
1,bu kadar mı hayır bu kadar değil,761,764,ölüm bir terhistir dedik terhis olan mutlu olu...,vOWbfQ0tIyU,https://www.youtube.com/watch?v=vOWbfQ0tIyU&t=...
2,var değil mi mi ben çok çok yok,84,86,ülkelerde yoğunluk var değil mi mi ben çok çok...,cUfp2lydmho,https://www.youtube.com/watch?v=cUfp2lydmho&t=84s
3,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc,https://www.youtube.com/watch?v=IGJVw1HjIHc&t=...
4,çok bir şey değil ama ben bunu,1663,1665,çok bir şey değil ama ben bunu almanı istiyorum,EtWqWE8gdF0,https://www.youtube.com/watch?v=EtWqWE8gdF0&t=...
...,...,...,...,...,...,...
195,için çok da bir şey de,203,205,de hani kamera çık olsa da olmasa da muhabbet ...,ag2ISgNp8fA,https://www.youtube.com/watch?v=ag2ISgNp8fA&t=...
196,yok yok ben bir daha o,6319,6322,yok yok ben bir daha o kızı göremem,s6vN0Sl2F1c,https://www.youtube.com/watch?v=s6vN0Sl2F1c&t=...
197,ama bu çok o kadar şey,900,901,ama bu çok o kadar şey yaptım,iSrsHDQASVs,https://www.youtube.com/watch?v=iSrsHDQASVs&t=...
198,da var bu bunu ben çok,1433,1435,bizim evde açsan yerin var abla gibi bir duygu...,xagFSGk8vT4,https://www.youtube.com/watch?v=xagFSGk8vT4&t=...


In [9]:
df_200 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
1,bilmiyorum sen ne demek istiyorsun ben de onu ...,721,726,bilmiyorum sen ne demek istiyorsun ben de onu ...,ZdeFof81xKs,https://www.youtube.com/watch?v=ZdeFof81xKs&t=...
2,var da bu bir sorun olur mu benim için ben bun...,7920,7926,var da bu bir sorun olur mu benim için ben bun...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
3,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...
4,gerçekten çok teşekkür ederim ya bu çok büyük ...,2257,2262,gerçekten çok teşekkür ederim ya bu çok büyük ...,a8Dov7B7alQ,https://www.youtube.com/watch?v=a8Dov7B7alQ&t=...
...,...,...,...,...,...,...
195,gerek yok çok teşekkür ederim ben zaten birkaç,186,190,gerek yok çok teşekkür ederim ben zaten birkaç...,zpfhmsZB26w,https://www.youtube.com/watch?v=zpfhmsZB26w&t=...
196,senin için ne demek olduğunu çok iyi biliyorum,7727,7730,polisliğin senin için ne demek olduğunu çok iy...,pWZnIAh9MXo,https://www.youtube.com/watch?v=pWZnIAh9MXo&t=...
197,ben gerçekten her şey için çok teşekkür ederim,875,878,ben gerçekten her şey için çok teşekkür ederim,d3LdJzLdpCI,https://www.youtube.com/watch?v=d3LdJzLdpCI&t=...
198,ki bu aslında bana iyi geldi çünkü o zaman şey,368,371,ki bu aslında bana iyi geldi çünkü o zaman şey...,X2DdZUtPDoU,https://www.youtube.com/watch?v=X2DdZUtPDoU&t=...


In [10]:
df_1000 = pd.read_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,https://www.youtube.com/watch?v=eqkDognLtTc&t=...
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
...,...,...,...,...,...,...
195,beni daha çok seviyor ben dedim ki hayır beni ...,368,373,o dedi ki resulullah beni daha çok seviyor ben...,yqj1HmYDs3w,https://www.youtube.com/watch?v=yqj1HmYDs3w&t=...
196,bilmiyorum sen ne demek istiyorsun ben de onu ...,721,726,bilmiyorum sen ne demek istiyorsun ben de onu ...,ZdeFof81xKs,https://www.youtube.com/watch?v=ZdeFof81xKs&t=...
197,hep aynı aman boş ver sen beni asıl sen nasıls...,2444,2450,hep aynı aman boş ver sen beni asıl sen nasıls...,m6k2GOeUHNI,https://www.youtube.com/watch?v=m6k2GOeUHNI&t=...
198,siz buradan artık yok olur musunuz lütfen bir ...,5000,5005,siz buradan artık yok olur musunuz lütfen bir ...,bt7g4VoVBoE,https://www.youtube.com/watch?v=bt7g4VoVBoE&t=...


#### Sending Data To Firestore

##### Absolute Beginner

In [11]:
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,ne o beni ne de ben onu bir daha,3870,3873,bundan böyle ne o beni ne de ben onu bir daha ...,FUS0hsPCtMs,https://www.youtube.com/watch?v=FUS0hsPCtMs&t=...
1,bu kadar mı hayır bu kadar değil,761,764,ölüm bir terhistir dedik terhis olan mutlu olu...,vOWbfQ0tIyU,https://www.youtube.com/watch?v=vOWbfQ0tIyU&t=...
2,var değil mi mi ben çok çok yok,84,86,ülkelerde yoğunluk var değil mi mi ben çok çok...,cUfp2lydmho,https://www.youtube.com/watch?v=cUfp2lydmho&t=84s
3,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc,https://www.youtube.com/watch?v=IGJVw1HjIHc&t=...
4,çok bir şey değil ama ben bunu,1663,1665,çok bir şey değil ama ben bunu almanı istiyorum,EtWqWE8gdF0,https://www.youtube.com/watch?v=EtWqWE8gdF0&t=...
...,...,...,...,...,...,...
195,için çok da bir şey de,203,205,de hani kamera çık olsa da olmasa da muhabbet ...,ag2ISgNp8fA,https://www.youtube.com/watch?v=ag2ISgNp8fA&t=...
196,yok yok ben bir daha o,6319,6322,yok yok ben bir daha o kızı göremem,s6vN0Sl2F1c,https://www.youtube.com/watch?v=s6vN0Sl2F1c&t=...
197,ama bu çok o kadar şey,900,901,ama bu çok o kadar şey yaptım,iSrsHDQASVs,https://www.youtube.com/watch?v=iSrsHDQASVs&t=...
198,da var bu bunu ben çok,1433,1435,bizim evde açsan yerin var abla gibi bir duygu...,xagFSGk8vT4,https://www.youtube.com/watch?v=xagFSGk8vT4&t=...


In [12]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [13]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description = {"description":"This description", "level":0, "levelName":"Absolute Beginner"}

In [14]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)

#beginner_data_list   

In [15]:
data_beginner = {"datas":{"0":beginner_description,"videos":beginner_data_list}}
#data_beginner

In [16]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [17]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [18]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description = {"description":"This description", "level":1, "levelName":"A1 Level"}

In [19]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [20]:
data_A1 = {"datas":{"1":A1_description,"videos":A1_data_list}}
#data_A1

##### A2 Level

In [21]:
# A1 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [22]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description = {"description":"This description", "level":2, "levelName":"A2 Level"}

In [23]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [24]:
data_A2 = {"datas":{"2":A2_description,"videos":A2_data_list}}
#data_A1

In [27]:
data_test = {"datas":[{"0":beginner_description,"videos":beginner_data_list},{"1":A1_description,"videos":A1_data_list},{"2":A2_description,"videos":A2_data_list}]}
data_test

{'datas': [{'0': {'description': 'This description',
    'level': 0,
    'levelName': 'Absolute Beginner'},
   'videos': [{'endTime': 3873, 'id': 'FUS0hsPCtMs', 'startTime': 3870},
    {'endTime': 764, 'id': 'vOWbfQ0tIyU', 'startTime': 761},
    {'endTime': 86, 'id': 'cUfp2lydmho', 'startTime': 84},
    {'endTime': 4268, 'id': 'IGJVw1HjIHc', 'startTime': 4267},
    {'endTime': 1665, 'id': 'EtWqWE8gdF0', 'startTime': 1663},
    {'endTime': 5168, 'id': 't4G3MB0ULX4', 'startTime': 5166},
    {'endTime': 2702, 'id': 'ZvpiV4iHzOE', 'startTime': 2698},
    {'endTime': 4188, 'id': 'aMRBsJyZFYc', 'startTime': 4186},
    {'endTime': 767, 'id': 'cQbynkEPQqs', 'startTime': 764},
    {'endTime': 6429, 'id': 'Q8KfV3zaCYQ', 'startTime': 6427},
    {'endTime': 397, 'id': 'U4CqkDvAE_Q', 'startTime': 394},
    {'endTime': 764, 'id': '_xkIGh8MnI0', 'startTime': 761},
    {'endTime': 3883, 'id': '5o2rhvamLBM', 'startTime': 3880},
    {'endTime': 92, 'id': 'EVexraDiTFc', 'startTime': 90},
    {'endTime': 

In [28]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_test)

update_time {
  seconds: 1669045832
  nanos: 28800000
}

In [ ]:
#data_all = {"datas":[{"0":beginner_description,"videos":beginner_data_list},{"1":A1_description,"videos":A1_data_list},{"2":A2_description,"videos":A2_data_list},\
#{"3":B1_description,"videos":B1_data_list},{"4":B2_description,"videos":B2_data_list},{"5":C1_description,"videos":C1_data_list},{"6":C2_description,"videos":C2_data_list}]}
##data_all  # will test

In [ ]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)  # will test

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000